# Hybrid Plant with Wind, Solar, Battery, Hydrogen and Ammonia Example
---
In this example we will simulate a hybrid plant using the GreenHEART software. This software allows for the co-design and simulation of an integrated renewable energy and ammonia production plant. 

This simulation starts with renewable energy generation using wind and solar PV technologies, which then can be stored in a battery or directly supplied to a PEM electrolyzer for hydrogen production. The produced hydrogen is then used as a feedstock in the ammonia model.

### Import Required Modules
---
Begin by importing the necessary modules for the GreenHEART simulation.

In [2]:
from greenheart.simulation.greenheart_simulation import GreenHeartSimulationConfig
from greenheart.tools.optimization.gc_run_greenheart import run_greenheart

### Load Simulation Input Files
---
Input files are in a YAML file format. Four YAML files are required for land-based analysis:
1. HOPP Configuration YAML
2. GreenHEART Configuration YAML
3. Wind Turbine Configuration YAML
4. FLORIS Configuration YAML

#### HOPP YAML Overview

The HOPP YAML file includes information required for simulating the renewable energy generation at the plant. Such as:

- **Analysis Location**: Specifies latitude, longitude, site boundaries, and resource files.
- **Generation Technologies**: Encompasses details on wind, solar PV, and battery storage systems.
- **General Generation Configuration Options**: Encompasses simulation options, battery dispatch strategies, and cost information.

In [3]:
filename_hopp_config = "./inputs/plant/hopp_config.yaml"

#### GreenHEART YAML Overview

The GreenHEART YAML file includes information required for simulating the renewable energy generation through the end product production (hydrogen, steel, ammonia). Such as:

- **Plant Design Scenarios**: Specifies location and connection of individual components of plant.
- **Financial Parameters**: General financial parameters needed for calculating the levelized cost of product and other financial metrics.
- **Hydrogen Equipment Information**: Including electrolyzer specifications, compressor, pipeline, and H<sub>2</sub> storage information.
- **Policy Options**: Describes options to include [IRA](https://www.whitehouse.gov/wp-content/uploads/2022/12/Inflation-Reduction-Act-Guidebook.pdf) tax incentives policy in GreenHEART simulation.

In [4]:
filename_greenheart_config = "./inputs/plant/greenheart_config_onshore-ammonia.yaml"

#### Wind Turbine and FLORIS YAML Overview

The GreenHEART simulation runs a steady-state wake modeling tool ([FLORIS](https://www.nrel.gov/wind/floris.html)) to assess the wind farm's energy production. For accurate modeling within FLORIS, two configuration YAML files are required:

- The **Wind Turbine YAML** file includes the description of the turbine including turbine rating rotor diameter, hub hight, and the power curve.
- The **FLORIS YAML** file includes atmospheric conditions and wake models used in simulation.

In [5]:
turbine_model = "lbw_6MW"
filename_turbine_config = "./inputs/turbines/" + turbine_model + ".yaml"
filename_floris_config = "./inputs/floris/floris_input_" + turbine_model + ".yaml"

### Create Full Configuration to Run GreenHEART Simulation
---
The class `GreenHeartSimulationConfig` reads in all the required YAML files described above, allows user to set additional settings while running the GreenHEART simulation, these include:
- **verbose**: If set to `True` additional information about the simulation will be printed as it runs.
- **show_plots**: If set to `True` the plots will pop up during the simulation.
- **save_plots**: If set to `True` the plots will be saved in the .output/figures folder.
- **use_profast**: [ProFAST](https://github.com/NREL/ProFAST) is a financial analysis software that provides a quick and convenient in-depth financial analysis for production systems and services based on Generally Accepted Accounting Principles (GAAP) methodology.
- **incentive_options**: Can select IRA incentives options which are located in the `GreenHEART YAML`.
- **plant_design_scenario**: Can select the plant design scenario from options which are located in the `GreenHEART YAML`.
- **output_level**: Determines what information is returned from the simulation.
- **post_processing**: If set to `True` the post_process_simulation() function in [utilities.py](https://github.com/NREL/GreenHEART/blob/main/greenheart/tools/eco/utilities.py) will be run. This function performs multiple post-processes dependent on the other arguments specified for the `GreenHeartSimulationConfig` class listed above and configuration set in the required YAML files.
    - If **verbose** is set to `True` additional information about the simulation will be printed.
    - If **show_plots** or **save_plots** are `True` additional plots of the plant will be shown or saved to ./output/figures folder.
    - If the `lca_config`:`run_lca` flag in the `GreenHEART YAML` is set to `True` Life Cycle Assessment (LCA) analysis will be run with the calculate_lca() function in [utilities.py](https://github.com/NREL/GreenHEART/blob/main/greenheart/tools/eco/utilities.py) and the data will be saved to .output/data/lca folder.

In [ ]:
config = GreenHeartSimulationConfig(
    filename_hopp_config,
    filename_greenheart_config,
    filename_turbine_config,
    filename_floris_config,
    verbose=True,
    show_plots=False,
    save_plots=False,
    use_profast=True,
    post_processing=True,
    incentive_option=1,
    plant_design_scenario=9,
    output_level=7,
)

### Run the GreenHEART Simulation
---
The simulation runs a techno-economic analysis of a given plant configuration. Data and figures are output to a seperate folder.

In [ ]:
# Suppress showing plots in output below if show_plots == False
if not config.show_plots:
    %matplotlib agg

# for analysis
prob, config = run_greenheart(config, run_only=True)

# for optimization
# prob, config = run_greenheart(config, run_only=False)

### Key Output Values

In [ ]:
lcoe = prob.get_val("lcoe", units="USD/(MW*h)")
lcoh = prob.get_val("lcoh", units="USD/kg")
lcoa = prob.get_val("lcoa", units="USD/kg")

print("LCOE: ", lcoe, "[$/MWh]")
print("LCOH: ", lcoh, "[$/kg]")
print("LCOA: ", lcoa, "[$/kg]")